Original Source: https://huggingface.co/blog/llama2

In [108]:
# !pip install --upgrade peft
# !pip install --upgrade transformers
!export PYTORCH_ENABLE_MPS_FALLBACK=1

In [109]:
from datasets import load_dataset
from peft import LoraConfig
from transformers import TrainingArguments, LlamaTokenizer, AutoModelForCausalLM
from trl import SFTTrainer

In [110]:
dataset_name = "timdettmers/openassistant-guanaco"
dataset_text_field = "text"
seq_length = 512
model_path = "/Users/bsantanna/dev/workspace/community/Llama-2-7b-chat-hf"

In [111]:
dataset = load_dataset(dataset_name, split="train")

Repo card metadata block was not found. Setting CardData to empty.


In [114]:
tokenizer = LlamaTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [115]:
training_args = TrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=1.41e-5,
    logging_steps=1,
    num_train_epochs=3,
    max_steps=-1,
    report_to=["wandb"],
    save_steps=100,
    save_total_limit=10,
    push_to_hub=False,
    hub_model_id=None,
)

peft_config = LoraConfig(
    r=64,
    lora_alpha=16,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    inference_mode=False
)

In [116]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    max_seq_length=seq_length,
    train_dataset=dataset,
    dataset_text_field=dataset_text_field,
    peft_config=peft_config,
)

In [ ]:
trainer.train()

Step,Training Loss
